In [1]:
from aocd import get_data, submit
import numpy as np 
data = get_data(year=2022, day=8).splitlines()
data = """30373
25512
65332
33549
35390""".splitlines()
data = [list(d) for d in data]
data = np.array(data).astype("int")
data

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 3, 3, 2],
       [3, 3, 5, 4, 9],
       [3, 5, 3, 9, 0]])

In [2]:
up_mask = np.maximum.accumulate(data, axis=0)
left_mask = np.maximum.accumulate(data, axis=1)
right_mask = np.fliplr(np.maximum.accumulate(np.fliplr(data), axis=1))
down_mask = np.flipud(np.maximum.accumulate(np.flipud(data), axis=0))
up_mask

array([[3, 0, 3, 7, 3],
       [3, 5, 5, 7, 3],
       [6, 5, 5, 7, 3],
       [6, 5, 5, 7, 9],
       [6, 5, 5, 9, 9]])

In [4]:
visible_up = np.diff(up_mask, axis=0, prepend=-1) > 0 
visible_right = np.fliplr(np.diff(np.fliplr(right_mask), axis=1, prepend=-1)) > 0 
visible_left = np.diff(left_mask, axis=1, prepend=-1) > 0 
visible_down = np.flipud(np.diff(np.flipud(down_mask), axis=0, prepend=-1)) > 0
visible_up.astype("int")

array([[1, 1, 1, 1, 1],
       [0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]])

In [136]:
visible = visible_up | visible_right | visible_left | visible_down
answer = np.sum(np.sum(visible))


In [137]:
submit(answer, year=2022, day=8, part="a")

Part a already solved with same answer: 1823


In [138]:
data

array([[2, 0, 2, ..., 1, 2, 0],
       [1, 1, 0, ..., 0, 1, 1],
       [0, 2, 2, ..., 1, 1, 2],
       ...,
       [2, 1, 1, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 2, 0],
       [1, 1, 0, ..., 2, 2, 0]])

In [6]:
N = 5
def view_count_N(data, direction, N):
    H, W = data.shape
    
    if direction == "up":
        rows, cols = range(H), range(W)
    elif direction == "down":
        rows = list(reversed(range(H)))
        cols = range(W)
    elif direction == "left":
        rows, cols = range(H), range(W)
    elif direction == "right":
        rows = range(H)
        cols = list(reversed(range(W)))
    
    count_view = np.zeros(data.shape).astype("int")
    if direction in ["up", "down"]:
        for col in cols:
            h = -1
            count = -1
            for row in rows:
                count += 1
                count_view[row, col] = count
                if data[row, col] >= N:
                    count = 0
        return count_view
    else:
        for row in rows:
            h = -1
            count = -1
            for col in cols:
                count += 1
                count_view[row, col] = count
                if data[row, col] >= N:
                    count = 0
        return count_view

view_count_N(data, "left", 5)

array([[0, 1, 2, 3, 1],
       [0, 1, 1, 1, 2],
       [0, 1, 1, 2, 3],
       [0, 1, 2, 1, 2],
       [0, 1, 1, 2, 1]])

In [140]:
def get_view_count_N(data, N):
    return (
        view_count_N(data, "left", N) * 
        view_count_N(data, "right", N) * 
        view_count_N(data, "down", N) * 
        view_count_N(data, "up", N) * 
        (data == N)
    )



best_count = 0
for n in range(1,10):
    if (best_for_N := np.max(np.max(get_view_count_N(data, n)))) > best_count:
        best_count = best_for_N
    print(n, best_for_N)
best_count

1 18
2 125
3 480
4 7623
5 13260
6 48960
7 211680
8 105280
9 96228


211680